In [ ]:
import os
import json
import pandas as pd
import numpy as np
import spacy
tokenizer = spacy.load('en')

In [ ]:
train_json_path = '../data/qangaroo_v1.1/wikihop/train.json'
dev_json_path = '../data/qangaroo_v1.1/wikihop/dev.json'

In [ ]:
def get_label(row):
    index = -1
    for i in range(len(row['candidates'])):
        c = row['candidates'][i]
        if c == row['answer']:
            index = i
            break
    return index


def get_mentions(item):
    candidates = item['candidates']
    supports = item['supports']

    candidates = [tokenizer(candidate.lower()) for candidate in candidates]
    supports = [tokenizer(support.lower()) for support in supports]

    all_mentions = []
    for candidate in candidates:
        mentions = []
        for idx, support in enumerate(supports):

            for i in range(len(support)):
                token = support[i]
                if token.text == candidate[0].text:
                    if support[i:i+len(candidate)].text == candidate.text:
                        mentions.append([idx, i, i+len(candidate)])
        all_mentions.append(mentions)
    return all_mentions

In [ ]:
df = pd.read_json(train_json_path)
df['query'] = df['query'].apply(lambda x: x.replace("_"," "))
df['label'] = df.apply(lambda x: get_label(x), axis=1)
df['mentions'] = df.apply(lambda x: get_mentions(x), axis=1)
df.to_json('../data/qangaroo_v1.1/wikihop/train_convert.json', orient='records')


df = pd.read_json(dev_json_path)
df['query'] = df['query'].apply(lambda x: x.replace("_"," "))
df['label'] = df.apply(lambda x: get_label(x), axis=1)
df['mentions'] = df.apply(lambda x: get_mentions(x), axis=1)
df.to_json('../data/qangaroo_v1.1/wikihop/dev_convert.json', orient='records')


In [ ]:
df = pd.read_json('../data/qangaroo_v1.1/wikihop/dev_convert.json')